In [ ]:
from src.drawProcessor     import drawProcessor
from src.drawTwoProcessors import drawTwoProcessors

## Example 1: 4 Types of Instructions

To simulate the processor, you need to provide a program with:

- 4 types of instructions: ALU, Branch, Load, NOP
- 8 registers: r0-r7
  - r0 is constant 0
  - r7 is also named as rSec

ROB has infinite entries so you do not need to worry about reuse of ROB entries. The format of 4 types of instruction are shown below

In [ ]:
drawProcessor(
    ## We have 4 types of instructions.
    imem=[
        {"dest": 0, "opcode": "ALU" , "src"   : 0, "port": 0, "latency": 1, "result": 0, "name": "xx"},
        {"dest": 0, "opcode": "LOAD", "src"   : 0,                                       "name": "xx"},
        {"dest": 0, "opcode": "LOAD", "srcImm": 1,                                       "name": "xx"},
        {           "opcode": "BREZ", "src"   : 0, "offset": 2,                          "name": "br"},
        {           "opcode": "NOP" ,                                                    "name": "nop"},
    ],
    r7=1,                                      ## Secret locates in r7
    l1ValidArray=[False, True, False, False],  ## We have 4 memory entries. This array determines which entry is pre-filled in L1
).getDraw().display_inline()

#### Code below shows all arguments you can use

In [ ]:
drawProcessor(
    ## We have 4 types of instructions.
    imem=[
        {"dest": 0, "opcode": "ALU" , "src"   : 0, "port": 0, "latency": 1, "result": 0, "name": "xx"},
        {"dest": 0, "opcode": "LOAD", "src"   : 0,                                       "name": "xx"},
        {"dest": 0, "opcode": "LOAD", "srcImm": 1,                                       "name": "xx"},
        {           "opcode": "BREZ", "src"   : 0, "offset": 2,                          "name": "br"},
        {           "opcode": "NOP" ,                                                    "name": "nop"},
    ],
    r7=1,                ## Secret locates in r7
    l1ValidArray=[False, True, False, False],  ## We have 4 memory entries. This array determines which entry is pre-filled in L1
    # defense="Baseline" ## What Spectre defense is added. Options are Baseline, InvisiSpec, GhostMinion.
    # maxCycle=None,     ## Total number of cycles being simulated. Use None to finish the imem. 
    # scale=1,           ## The size of the figure
    # xyRatio=8/3,       ## The width / height ratio of the figure
    # speed=0.5,         ## Simulation speed, the larger the faster
).getDraw().display_inline()
# ).save("animation")    ## To save an animation.svg file, replace the last line with this line.

## Example 2: Trigger ALU Port Contention

ALU has 4 ports, operating in parallel. Each port has a FIFO to serve requests one-by-one.

In [ ]:
drawProcessor(
    imem=[
        {"dest": 0, "opcode": "ALU" , "src"   : 0, "latency": 1, "port": 0, "result": 0, "name": "0"},
        {"dest": 0, "opcode": "ALU" , "src"   : 0, "latency": 2, "port": 0, "result": 0, "name": "1"},
        {"dest": 0, "opcode": "ALU" , "src"   : 0, "latency": 4, "port": 0, "result": 0, "name": "2"},
        {"dest": 0, "opcode": "ALU" , "src"   : 0, "latency": 4, "port": 0, "result": 0, "name": "3"},
        {"dest": 0, "opcode": "ALU" , "src"   : 0, "latency": 1, "port": 2, "result": 0, "name": "4"},
        {"dest": 0, "opcode": "ALU" , "src"   : 0, "latency": 1, "port": 0, "result": 0, "name": "5"},
    ],
    r7=1,
    l1ValidArray=[False, False, False, False],
).getDraw().display_inline()

## Example 3: Cache Hit/Miss, and Use MSHR

Memory has 4 entries.
  - L1 can cache all 4 entries. Hit takes 1 cycle. Use `l1ValidArray` argument to determine which entry is pre-filled in L1.
  - On L1 miss, wait 3 cycles in MSHR

In [ ]:
drawProcessor(
    imem=[
        {"dest": 1, "opcode": "LOAD", "src"   : 0, "name": "0"},
        {"dest": 0, "opcode": "LOAD", "src"   : 1, "name": "1"},
        {"dest": 0, "opcode": "LOAD", "srcImm": 3, "name": "2"},
        {"dest": 1, "opcode": "LOAD", "srcImm": 3, "name": "3"},
        {"dest": 0, "opcode": "LOAD", "srcImm": 3, "name": "4"},
        {"dest": 0, "opcode": "LOAD", "srcImm": 2, "name": "5"},
        {           "opcode": "NOP" ,              "name": "6"},
    ],
    r7=1,
    l1ValidArray=[False, False, False, False],
).getDraw().display_inline()

## Example 4: Use Branch

In [ ]:
drawProcessor(
    imem=[
        {"dest": 1, "opcode": "LOAD", "src": 0, "latency": 1, "port": 0, "name": "0"},
        {           "opcode": "BREZ", "src": 1, "offset": 5,             "name": "br"},
        {"dest": 0, "opcode": "LOAD", "src": 1,                          "name": "2"},
        {"dest": 0, "opcode": "LOAD", "src": 0,                          "name": "3"},
        {"dest": 1, "opcode": "LOAD", "src": 7,                          "name": "4"},
        {"dest": 0, "opcode": "LOAD", "src": 0,                          "name": "5"},
        {           "opcode": "NOP" ,                                    "name": "nop"},
    ],
    r7=1,
    l1ValidArray=[False, False, False, False],
).getDraw().display_inline()

## Example 5: A simple Spectre attack

In [ ]:
Spectre = [
    {"dest": 1, "opcode": "ALU" , "src"   : 0, "port": 0, "latency": 4, "result": 0, "name": "delay"},
    {           "opcode": "BREZ", "src"   : 1, "offset": 6,                          "name": "br"},
    {"dest": 0, "opcode": "LOAD", "src"   : 7,                                       "name": "Tx"},
    {           "opcode": "NOP" ,                                                    "name": "nop"},
    {           "opcode": "NOP" ,                                                    "name": "nop"},
    {           "opcode": "NOP" ,                                                    "name": "nop"},
    {           "opcode": "NOP" ,                                                    "name": "nop"},
    {"dest": 0, "opcode": "LOAD", "srcImm": 0,                                       "name": "Rx"},
]
drawProcessor(
    imem=Spectre,
    r7=0,
    l1ValidArray=[False, False, False, False],
).getDraw().display_inline()

## Example 5: Show 2 Simulations Together with Different Secret Values

In [ ]:
drawTwoProcessors(
    imem=Spectre,
    r7Pair=(0, 1),
    l1ValidArray=[False, False, False, False],
).getDraw().display_inline()